<a href="https://colab.research.google.com/github/ibitoladgr8/Machine-Learning/blob/main/Forecasting_Sticker_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [80]:
train_df=pd.read_csv('/content/train.csv')
test_df=pd.read_csv('/content/test.csv')

In [81]:
train_df.isnull().sum()

,0
id,0
date,0
country,0
store,0
product,0
num_sold,8871


In [83]:
def extract_date_components(df):

  df['date'] = pd.to_datetime(df['date'])  # Convert 'date' column to datetime format

  df['year'] = df['date'].dt.year
  df['month'] = df['date'].dt.month_name()
  df['day'] = df['date'].dt.day
  df['day_of_week'] = df['date'].dt.day_name()
  df['week_of_year'] = df['date'].dt.isocalendar().week

  return df

# Apply the function to your training and test DataFrames
extract_date_components(train_df)
extract_date_components(test_df)

,id,date,country,store,product,year,month,day,day_of_week,week_of_year
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose,2017,January,1,Sunday,52
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle,2017,January,1,Sunday,52
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers,2017,January,1,Sunday,52
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler,2017,January,1,Sunday,52
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode,2017,January,1,Sunday,52
...,...,...,...,...,...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose,2019,December,31,Tuesday,1
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle,2019,December,31,Tuesday,1
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2019,December,31,Tuesday,1
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler,2019,December,31,Tuesday,1


In [84]:
import pandas as pd
import holidays

def get_holiday_name(date):

  try:
    us_holidays = holidays.US()
    if date in us_holidays:
      return us_holidays.get(date)
    else:
      return None
  except ValueError:
    return None

def create_holiday_name_column(df, date_column):

  df['holiday_name'] = df[date_column].apply(get_holiday_name)
  return df


create_holiday_name_column(train_df, 'date')
create_holiday_name_column(test_df, 'date')

,id,date,country,store,product,year,month,day,day_of_week,week_of_year,holiday_name
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose,2017,January,1,Sunday,52,New Year's Day
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle,2017,January,1,Sunday,52,New Year's Day
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers,2017,January,1,Sunday,52,New Year's Day
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler,2017,January,1,Sunday,52,New Year's Day
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode,2017,January,1,Sunday,52,New Year's Day
...,...,...,...,...,...,...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose,2019,December,31,Tuesday,1,None
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle,2019,December,31,Tuesday,1,None
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2019,December,31,Tuesday,1,None
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler,2019,December,31,Tuesday,1,None


In [85]:
categorical_cols = [cname for cname in train_df if train_df[cname].dtype=='object']
categorical_cols

['country', 'store', 'product', 'month', 'day_of_week', 'holiday_name']

In [105]:
train_df['holiday_name'].unique()


array(["New Year's Day", 'No Holiday', 'Martin Luther King Jr. Day',
       "Washington's Birthday", 'Memorial Day', 'Independence Day',
       'Independence Day (observed)', 'Labor Day', 'Columbus Day',
       'Veterans Day', 'Thanksgiving', 'Christmas Day (observed)',
       'Christmas Day', "New Year's Day (observed)",
       'Veterans Day (observed)'], dtype=object)

In [102]:
train_df.drop('date', axis=1, inplace = True)
test_df.drop('date', axis=1, inplace = True)

In [104]:
train_df['holiday_name'].fillna('No Holiday', inplace=True)
test_df['holiday_name'].fillna('No Holiday', inplace=True)

In [106]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
train_df['num_sold'] = imputer.fit_transform(train_df[['num_sold']])

In [107]:
train_df.isnull().sum()

,0
id,0
country,0
store,0
product,0
num_sold,0
year,0
month,0
day,0
day_of_week,0
week_of_year,0


In [108]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in categorical_cols:
  train_df[col] = label_encoder.fit_transform(train_df[col])
  test_df[col] = label_encoder.transform(test_df[col])

In [71]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

In [109]:
X= train_df.drop(['id','num_sold'], axis=1)
y= train_df['num_sold']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=42)

In [110]:
from sklearn.metrics import mean_absolute_error
model1 = RandomForestRegressor(n_estimators= 100,
                               max_depth =10,
                               min_samples_split=2,
                               min_samples_leaf=1
                              )

model1.fit(X_train, y_train)
pred1 =model1.predict(X_valid)

score1 = mean_absolute_error(y_valid, pred1)
print('MAE:', score1)


MAE: 63.01180590547116


In [111]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
model2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

model2.fit(X_train, y_train)
pred2 =model2.predict(X_valid)

score2 = mean_absolute_error(y_valid, pred2)
print('MAE:', score2)

MAE: 42.51702705868618


In [114]:
X_valid.shape[1], test_df.shape[1]

(9, 10)

In [113]:
test_df.columns, X_train.columns

(Index(['id', 'country', 'store', 'product', 'year', 'month', 'day',
        'day_of_week', 'week_of_year', 'holiday_name'],
       dtype='object'),
 Index(['country', 'store', 'product', 'year', 'month', 'day', 'day_of_week',
        'week_of_year', 'holiday_name'],
       dtype='object'))

In [115]:
test_df_to_use = test_df.drop(['id'], axis=1)
test_df_to_use.shape, test_df.shape

((98550, 9), (98550, 10))

In [79]:
prediction = model1.predict(test_df_to_use)
prediction_xgb = model2.predict(test_df_to_use)

submissionrf = pd.DataFrame({'id': test_df['id'], 'num_sold': prediction})
submissionrf.to_csv('submission3.csv', index=False)

submissionxgb =pd.DataFrame({'id': test_df['id'], 'num_sold': prediction_xgb})
submissionxgb.to_csv('submission4.csv', index=False)

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import RootMeanSquaredError

In [22]:
X_train['week_of_year'] = X_train['week_of_year'].astype(int)
X_valid['week_of_year'] = X_valid['week_of_year'].astype(int)

In [25]:
for col in X_train.select_dtypes(include=['number']).columns:
    if X_train[col].dtype != np.float32 and X_train[col].dtype != np.float64:
        X_train[col] = X_train[col].astype(np.float32)  # or np.float64
        X_valid[col] = X_valid[col].astype(np.float32)

In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]), # Updated input shape
    tf.keras.layers.Dropout(0.2),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Re-compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[RootMeanSquaredError()])


In [27]:
from sklearn.metrics import mean_absolute_error

X_train = X_train.astype(np.float32)  # or np.float64
y_train = y_train.astype(np.float32)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=100,
    min_delta=0.001,
    restore_best_weights=True,
)

model.fit(
X_train, y_train,
    batch_size=512,  # Add batch size
    epochs=1000,      # Add epochs
    callbacks=[early_stopping],  # Add callbacks
    verbose=0         # Add verbose
)



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [28]:
X_valid= X_valid.astype(np.float32)  # or np.float64
y_valid= y_valid.astype(np.float32)

preds = model.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

1439/1439 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
MAE: 752.865966796875


In [ ]:
 test_df_to_use = test_df.drop('id', axis=1)

 test_df_to_use = scaler.transform(test_df_to_use)

 predictions = model.predict(test_df_to_use)

 submission_df = pd.DataFrame({'id': test_df['id'], 'num_sold': predictions.flatten()})
 submission_df.to_csv('submission.csv', index=False)

3080/3080 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step
